In [1]:
import pandas as pd

from tqdm import tqdm

import matplotlib.pyplot as plt
import lib.draw as draw
from lib.pitch_control import plot_pitch_control
import imageio
import os

In [2]:
df_grids = pd.read_csv('res/grids.csv')
df_grids.head(4)

,play_frame_id,model_name,c0,c1,c2,c3,c4,c5,c6,c7,...,c7304,c7305,c7306,c7307,c7308,c7309,c7310,c7311,c7312,c7313
0,1,KNN (Voronoi),1.00,1.00,1.00,1.00,-1.00,-1.00,-1.00,-1.00,...,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00
1,1,KNN (Spearman),1.00,0.60,0.33,0.14,0.00,-0.25,-0.50,-0.75,...,-0.75,-0.83,-0.92,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00
2,1,KNN (Fernandez),0.07,0.06,0.04,0.02,-0.00,-0.02,-0.04,-0.07,...,-0.15,-0.16,-0.17,-0.18,-0.18,-0.19,-0.19,-0.19,-0.19,-0.19
3,1,Spearman Pitch Control,0.11,0.10,0.09,0.09,0.08,0.07,0.06,0.05,...,0.29,0.27,0.25,0.23,0.22,0.20,0.19,0.18,0.16,0.14


In [3]:
df_results = pd.read_csv('res/results.csv')
df_results.head(4)

,play_frame_id,model,play,frame,inference_time
0,1,KNN (Voronoi),Liverpool [2] - 0 Porto,0.0,0.222970
1,1,KNN (Spearman),Liverpool [2] - 0 Porto,0.0,0.764854
2,1,KNN (Fernandez),Liverpool [2] - 0 Porto,0.0,0.779044
3,1,Spearman Pitch Control,Liverpool [2] - 0 Porto,0.0,33.991668


In [4]:
df_data = pd.read_csv('data/all_goals.csv', low_memory=False)
df_data.edgecolor.fillna('white', inplace=True)
df_data.bgcolor.fillna('black', inplace=True)
df_data.head(4)

,bgcolor,dx,dy,edgecolor,frame,play,player,player_num,team,x,y,z,coords,player_obj,num,name,play_id
0,black,0.000000,0.000000,white,0,Liverpool [3] - 0 Bournemouth,0,NaN,NaN,46.394558,11.134454,0.0,NaN,NaN,NaN,NaN,15
1,black,0.185745,1.217580,white,1,Liverpool [3] - 0 Bournemouth,0,NaN,NaN,46.580302,12.352034,0.0,NaN,NaN,NaN,NaN,15
2,black,0.178659,1.171133,white,2,Liverpool [3] - 0 Bournemouth,0,NaN,NaN,46.758961,13.523166,0.0,NaN,NaN,NaN,NaN,15
3,black,0.171573,1.124685,white,3,Liverpool [3] - 0 Bournemouth,0,NaN,NaN,46.930535,14.647852,0.0,NaN,NaN,NaN,NaN,15


In [5]:
def generate_gif(grids, results, player_data, play_name, model_name):
    _grids = grids[(grids.model_name == model_name) & grids.play_frame_id.isin(results[results.play == play_name].play_frame_id)]
    grid = pd.DataFrame([[i/1.05, j/0.68] for i in range(106) for j in range(69)], columns=['x','y'])
    temp_file_name = '_tmp/104.png'

    writer = imageio.get_writer(f'img/{model_name}_{play_name}.gif', mode='I')
    for original_frame_no, frame_no in tqdm(enumerate(_grids.play_frame_id.unique()), total=len(_grids.play_frame_id.unique())):
        _frame = _grids[_grids.play_frame_id == frame_no]
        original_frame = player_data[(player_data.frame == original_frame_no) & (player_data.play == play_name)]
        plot_pitch_control(original_frame, grid, _frame.iloc[:, 2:].to_numpy(), modelname=model_name, savefig=temp_file_name)
        image = imageio.imread(temp_file_name)
        writer.append_data(image)
    writer.close()

In [6]:
for i, row in df_results[['model', 'play']].drop_duplicates().iterrows():
    if os.path.isfile(f'img/{row.model}_{row.play}.gif') == False:
        generate_gif(df_grids, df_results, df_data, row.play, row.model)

100%|██████████| 289/289 [02:01<00:00,  2.38it/s]


<Figure size 1260x816 with 0 Axes>